<a href="https://colab.research.google.com/github/Zooshi/HerokuDBZ/blob/master/notebookeb1879d3f0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv('/kaggle/input/toxiccomments/toxic comments.csv')
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForSequenceClassification
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [ ]:
comments = df['comment_text'].tolist()
train = comments[:10000]
test = comments[10000:20000]

In [ ]:
train_tk = tokenizer(train,padding='max_length',max_length = 512, truncation =True, return_tensors='pt')

In [ ]:
test_tk = tokenizer(test,padding='max_length', max_length=512,truncation=True,return_tensors='pt')

In [ ]:
train_tk.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
train_tk['input_ids'][0].dtype

torch.int64

In [ ]:
# train_tk['input_ids'][0]
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

y = torch.tensor(df.iloc[:,2:].values)


In [ ]:
y_train = y[:10000]
y_test = y[10000:20000]
assert len(y_train)== len(y_test)

In [ ]:
class Dataset:
    
    def __init__(self,encodings,labels):
        self.encodings = encodings
        self.labels = labels
        
    
    def __len__(self):
        return len(self.labels)
        
    def __getitem__(self,idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask':self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }

In [ ]:
train_ds = Dataset(train_tk,y_train)
test_ds = Dataset(test_tk,y_test)

In [ ]:
# train_ds[42]
train_dl = DataLoader(train_ds,batch_size=4, shuffle=True)
test_dl = DataLoader(test_ds,batch_size=4)

In [ ]:
ex = next(iter(train_dl))

In [ ]:
device = ('cuda' if torch.cuda.is_available else 'cpu')
device

'cuda'

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.bert = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
        self.final_layer = self.bert.config.hidden_size
        self.final = nn.Linear(self.final_layer,6)
        self.bert.classifier = self.final
        
    def forward(self,encodings):
        input_ids = encodings['input_ids'].to(device)
        attention_mask = encodings['attention_mask'].to(device)
        labels  = encodings['labels'].to(device)
        output = self.bert(input_ids,attention_mask)
        return output

In [ ]:
model = Model()
loss = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_clas

In [ ]:
#erst mal ein Beispiel um zu testen ob ein Output rauskommt
#nach cuda error mal cpu probieren
#device = 'cpu'
model = model.to(device)
result =model(ex)

In [ ]:
#Float conversion + device, hat funktioniert!
l = loss(result['logits'],ex['labels'].type(torch.FloatTensor).to(device))
l

tensor(0.7079, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [ ]:
#result['logits']
torch.cuda.empty_cache()

In [ ]:
for i in range(3):
    epoch_loss = 0.0
    for i, batch in enumerate(train_dl):
        #model = model.to(device)
        labels = batch['labels'].type(torch.FloatTensor).to(device)
        output = model(batch)
        optimizer.zero_grad()
        batch_loss = loss(output['logits'],labels)
        epoch_loss += batch_loss.item()
        batch_loss.backward()
        optimizer.step()
        if i%10 ==0:
            print('current loss',batch_loss.item())
    print('epoch_done, loss',epoch_loss)